# Example: Time-Series Analysis

This notebook demonstrates time-series processing and analysis for land clearing detection.

In [ ]:
# Import required modules
from aus_land_clearing.data import load_dea_fractional_cover
from aus_land_clearing.processing import (
    extract_time_series,
    calculate_change_statistics,
    detect_clearing_events,
    aggregate_by_period,
    calculate_vegetation_indices
)
from aus_land_clearing.utils import get_study_area_bbox, get_time_range
import matplotlib.pyplot as plt

## 1. Load Data

In [ ]:
# Define study parameters
bbox = get_study_area_bbox('queensland')
start_year, end_year = get_time_range()
time_range = (f'{start_year}-01-01', f'{end_year}-12-31')

# Load fractional cover data
fc_data = load_dea_fractional_cover(
    bbox=bbox,
    time_range=time_range,
    bands=['PV', 'NPV', 'BS']
)

print(f"Data loaded: {fc_data}")

## 2. Calculate Vegetation Indices

In [ ]:
# Calculate additional vegetation indices
fc_indices = calculate_vegetation_indices(fc_data)

print("Available variables:")
for var in fc_indices.data_vars:
    print(f"  - {var}")

## 3. Extract Time Series

In [ ]:
# Extract time series for photosynthetic vegetation
pv_timeseries = extract_time_series(
    fc_indices,
    variable='PV',
    method='mean'
)

print("Time series extracted:")
print(pv_timeseries.head())

# Extract time series for vegetation health
health_timeseries = extract_time_series(
    fc_indices,
    variable='vegetation_health',
    method='mean'
)

print("\nVegetation health time series:")
print(health_timeseries.head())

## 4. Calculate Change Statistics

In [ ]:
# Compare baseline (1988-1995) to recent (2015-2024)
change_stats = calculate_change_statistics(
    fc_data,
    variable='PV',
    baseline_period=('1988-01-01', '1995-12-31'),
    comparison_period=('2015-01-01', '2024-12-31')
)

print("Change Statistics:")
print(f"Baseline mean: {change_stats.get('baseline_mean', 'N/A'):.2f}")
print(f"Comparison mean: {change_stats.get('comparison_mean', 'N/A'):.2f}")
print(f"Absolute change: {change_stats.get('absolute_change', 'N/A'):.2f}")
print(f"Percent change: {change_stats.get('percent_change', 'N/A'):.2f}%")

## 5. Aggregate by Time Period

In [ ]:
# Aggregate to annual resolution
annual_data = aggregate_by_period(
    fc_data,
    variable='PV',
    period='year'
)

print("Annual aggregation complete")
print(f"Original temporal resolution: {len(fc_data.time) if 'time' in fc_data.dims else 0} timesteps")
print(f"Annual resolution: {len(annual_data.time) if 'time' in annual_data.dims else 0} years")

## 6. Detect Clearing Events

In [ ]:
# Detect clearing events (>25% decline in vegetation)
clearing_events = detect_clearing_events(
    fc_data,
    variable='PV',
    threshold=-25.0,
    min_duration=1
)

print(f"Detected {len(clearing_events)} clearing events")
if len(clearing_events) > 0:
    print("\nFirst few events:")
    print(clearing_events.head())

## Next Steps

- See `03_visualization.ipynb` for creating visualizations from this analysis
- See `04_narrative_stories.ipynb` for story-driven visualizations